In [ ]:
import re

with open('input.txt', 'r') as f:
  lines = f.read().splitlines()

In [ ]:
def get_seeds(lines):
  m = re.findall(r'(\d+)\s+(\d+)', lines[0])
  return list(map(lambda x: (int(x[0]), int(x[1])), m))

def get_section(section_name, lines):
  section_lines = []
  in_section = False

  # Load lines if section
  for i in range(len(lines)):
    if lines[i] == section_name:
      in_section = True
    elif in_section:
      if lines[i] == '':
        break
      else:
        section_lines.append(lines[i])

  # Convert section lines to numbers
  return [list(map(int, re.split(r'\s+', l))) for l in section_lines]

maps = [
  get_section('seed-to-soil map:', lines),
  get_section('soil-to-fertilizer map:', lines),
  get_section('fertilizer-to-water map:', lines),
  get_section('water-to-light map:', lines),
  get_section('light-to-temperature map:', lines),
  get_section('temperature-to-humidity map:', lines),
  get_section('humidity-to-location map:', lines)
]

In [ ]:
# Transform a single point via a given map
def map_point(k, map):
  for j in range(len(map)):
    [dest, src, size] = map[j]
    if k >= src and k < src + size:
      k = dest + (k - src)
      break
  return k

# Transform a (start, size) range via a given map
def map_range(r, map):
  (k, size) = r
  k_prime = map_point(k, map)
  return (k_prime, size)

def get_split_points(map):
  split_at = []
  for i in range(len(map)):
    [dest, src, size] = map[i]
    split_at.append(src)
    split_at.append(src + size)
  return list(sorted(set(split_at)))

def split_range(r, split_at):
  (r_start, r_size) = r
  #print(f'split_range: r = {r}, split_at = {split_at}')
  split_points_in_range = [k for k in split_at if k >= r_start and k < r_start + r_size]
  #print(f'  split_points_in_range: {split_points_in_range}')

  rs = []
  for sp in split_points_in_range:
    (r_start, r_size) = r
    rs.append((r_start, sp - r_start))
    r = (sp, r_size - (sp - r_start))

  rs.append(r)

  #print(f'  => rs: {rs}')
  return rs

# Transform a [(start, size), ...] list via a given map
def map_ranges(rs, map):
  split_at = get_split_points(map)
  rs = [rs_prime
        for r in rs
        for rs_prime in split_range(r, split_at)]
  rs = [map_range(r, map) for r in rs]
  return rs

def map_all_seed_ranges(rs, maps):
  for i in range(len(maps)):
    rs = map_ranges(rs, maps[i])
  return rs

seeds = get_seeds(lines)
resulting_ranges = map_all_seed_ranges(seeds, maps)
lowest_result = min(resulting_ranges, key=lambda x: x[0])
lowest_result